In [13]:
import asyncio
from typing import List
import httpx
from walkscore_api import WalkScore
import json
from parsel import Selector
import pandas as pd

ImportError: cannot import name '_get_object_size' from 'bson' (c:\Users\Admin\miniconda3\envs\recsys\Lib\site-packages\bson\__init__.py)

In [ ]:
async def scrape_properties(urls: List[str], client):

    to_scrape = [client.get(url) for url in urls]
    results = []

    for response in asyncio.as_completed(to_scrape):
        response = await response
        assert response.status_code == 200, "request has been blocked"
        selector = Selector(response.text)
        data = selector.css("script#__NEXT_DATA__::text").get()
        if data:
            # Option 1: some properties are located in NEXT DATA cache
            data = json.loads(data)

            results_temp = data["props"]["pageProps"]["searchPageState"]["cat1"]["searchResults"]["listResults"]
            results.extend(results_temp)
    
    return results

In [ ]:
client = httpx.AsyncClient(
    # enable http2
    http2=True,
    # add basic browser like headers to prevent being blocked
    headers={
        "accept-language": "en-US,en;q=0.9",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "accept-language": "en-US;en;q=0.9",
        "accept-encoding": "gzip, deflate, br",
    },
)

urls = ['https://www.zillow.com/los-angeles-ca/']

In [ ]:
results = await scrape_properties(urls, client)

In [ ]:
df = pd.DataFrame(results)
df['addressCity']

0         Studio City
1         Los Angeles
2       Beverly Hills
3        Sherman Oaks
4       Granada Hills
5              Sylmar
6             Sunland
7          Chatsworth
8       Mission Hills
9      Woodland Hills
10        Los Angeles
11        Los Angeles
12        Los Angeles
13     Woodland Hills
14         Wilmington
15    North Hollywood
16    North Hollywood
17          San Pedro
18     Woodland Hills
19      Playa Del Rey
20        Canoga Park
21        North Hills
22        Los Angeles
23        Los Angeles
24     Woodland Hills
25        Los Angeles
26         Northridge
27        Los Angeles
28        Studio City
29       Porter Ranch
30        Los Angeles
31      Mission Hills
32     Woodland Hills
33          San Pedro
34     Woodland Hills
35        Los Angeles
36      Beverly Hills
37        Los Angeles
38             Encino
39        Los Angeles
40         Sun Valley
Name: addressCity, dtype: object

In [ ]:
df.to_csv("./Data/LA.csv", index=False)

In [ ]:
# Create an instance of WalkScore
ws = WalkScore()

# Load your dataset into a pandas DataFrame
# Assuming df contains your dataset
# Replace this with loading your actual dataset
df = pd.read_csv('./Data/LA.csv')

# Filter the DataFrame to include only listings in Los Angeles
# df_la = df[df['city'] == 'Los Angeles']

# Query the Walk Score API for Los Angeles listings
ws.query_api(df)

# Get and format the data into a DataFrame
ws.get_and_format_data()

# Display the resulting DataFrame
print(ws.df_api)


NameError: name 'WalkScore' is not defined